In [34]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("/Users/mavingill/Downloads/Starter_Code-4/PyCitySchools/Resources/schools_complete.csv")
student_data_to_load = Path("/Users/mavingill/Downloads/Starter_Code-4/PyCitySchools/Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()


#create array of unique school names
unique_school_names = school_data['school_name'].unique()

#gives the length of unique school names to give us how many schools
school_count = len(unique_school_names)

#district student count
dist_student_count = school_data['size'].sum()

#student count from student file (to verify with district student count)
total_student_rec = student_data['student_name'].count()

#total budget
total_budget = school_data['budget'].sum()

#calculations for number and % passing reading
num_passing_reading = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = num_passing_reading/total_student_rec

#calculations for number and % passing math
num_passing_math = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
perc_pass_math = num_passing_math/total_student_rec 

#average math score calculation
avg_math_score = student_data['math_score'].mean()

#average reading score calculation
avg_reading_score = student_data['reading_score'].mean()

#Overall Passing Rate Calculations
overall_pass = student_data[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]['student_name'].count()/total_student_rec 

# district dataframe from dictionary

district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [dist_student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[perc_pass_reading],
    "% Passing Math": [perc_pass_math],
    "Overall Passing Rate": [overall_pass]

})

#store as different df to change order
dist_sum = district_summary[["Total Schools", 
                             "Total Students", 
                             "Total Budget", 
                             "Average Reading Score", 
                             "Average Math Score", 
                             '% Passing Reading', 
                             '% Passing Math', 
                             'Overall Passing Rate']]

#format cells
dist_sum.style.format({"Total Budget": "${:,.2f}", 
                       "Average Reading Score": "{:.6f}", 
                       "Average Math Score": "{:.6f}", 
                       "% Passing Math": "{:.6%}", 
                       "% Passing Reading": "{:.6%}", 
                       "Overall Passing Rate": "{:.6%}"})



,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,"$24,649,428.00",81.877840,78.985371,85.805463%,74.980853%,65.172326%


In [36]:
#groups by school
by_school = school_data_complete.set_index('school_name').groupby(['school_name'])

#school types
sch_types = school_data.set_index('school_name')['type']

# total students by school
stu_per_sch = by_school['Student ID'].count()

# school budget
sch_budget = school_data.set_index('school_name')['budget']

#per student budget
stu_budget = school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size']

#avg scores by school
avg_math = by_school['math_score'].mean()
avg_read = by_school['reading_score'].mean()

# % passing scores
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_per_sch 

sch_summary = pd.DataFrame({
    "School Type": sch_types,
    "Total Students": stu_per_sch,
    "Per Student Budget": stu_budget,
    "Total School Budget": sch_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})




#munging
sch_summary = sch_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

#formatting
sch_summary.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,.2f}", 
                          "Per Student Budget": "${:,.2f}",
                          'Average Math Score': "{:.6f}", 
                          'Average Reading Score': "{:.6f}", 
                          "% Passing Math": "{:.6%}", 
                          "% Passing Reading": "{:.6%}", 
                          "Overall Passing Rate": "{:.6%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064%,81.933280%,54.642283%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477%,97.039828%,91.334769%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471%,80.739234%,53.204476%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602%,79.299014%,54.289887%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371%,97.138965%,90.599455%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967%,80.862999%,53.527508%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855%,96.252927%,89.227166%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922%,81.316421%,53.513884%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551%,81.222432%,53.539172%


In [6]:
# sort values by passing rate and then only print top 5 
top_5 = sch_summary.sort_values("Overall Passing Rate", ascending = False)
top_5.head().style.format({'Total Students': '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}", 
                           "% Passing Math": "{:.1%}", 
                           "% Passing Reading": "{:.1%}", 
                           "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.061895,83.975780,94.1%,97.0%,91.3%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.418349,83.848930,93.3%,97.3%,90.9%
Griffin High School,Charter,"1,468","$917,500",$625,83.351499,83.816757,93.4%,97.1%,90.6%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.274201,83.989488,93.9%,96.5%,90.6%
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.6%,95.9%,90.5%


In [7]:
#bottom 5 schools from worse to best
#take tail of top5 sort and re-sort from worst to best
bottom_5 = top_5.tail()
bottom_5 = bottom_5.sort_values('Overall Passing Rate')
bottom_5.style.format({'Total Students': '{:,}', 
                       "Total School Budget": "${:,}", 
                       "Per Student Budget": "${:.0f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.842711,80.744686,66.4%,80.2%,53.0%
Figueroa High School,District,"2,949","$1,884,411",$639,76.711767,81.158020,66.0%,80.7%,53.2%
Huang High School,District,"2,917","$1,910,635",$655,76.629414,81.182722,65.7%,81.3%,53.5%
Hernandez High School,District,"4,635","$3,022,020",$652,77.289752,80.934412,66.8%,80.9%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.072464,80.966394,66.1%,81.2%,53.5%


In [8]:
#creates grade level average math scores for each school 
ninth_math = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School"

#show and format
math_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


In [9]:
#creates grade level average reading scores for each school
ninth_reading = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_reading = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_reading = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_reading = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

#merges the reading score averages by school and grade together
reading_scores = pd.DataFrame({
        "9th": ninth_reading,
        "10th": tenth_reading,
        "11th": eleventh_reading,
        "12th": twelfth_reading
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School"

#format
reading_scores.style.format({'9th': '{:.1f}', 
                             "10th": '{:.1f}', 
                             "11th": "{:.1f}", 
                             "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


In [10]:
# create spending bins
bins = [0, 584.999, 614.999, 644.999, 999999]
group_name = ['< $585', "$585 - 614", "$615 - 644", "> $644"]
school_data_complete['spending_bins'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], bins, labels = group_name)

#group by spending
by_spending = school_data_complete.groupby('spending_bins')

#calculations
avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()

            
# df build            
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_spend.index.name = "Per Student Budget"
scores_by_spend = scores_by_spend.reindex(group_name)

#formating
scores_by_spend.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
< $585,83.4,84.0,93.7%,96.7%,90.6%
$585 - 614,83.5,83.8,94.1%,95.9%,90.1%
$615 - 644,78.1,81.4,71.4%,83.6%,60.3%
> $644,77.0,81.0,66.2%,81.1%,53.5%


In [11]:
# create size bins
bins = [0, 999, 1999, 99999999999]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
school_data_complete['size_bins'] = pd.cut(school_data_complete['size'], bins, labels = group_name)

#group by spending
by_size = school_data_complete.groupby('size_bins')

#calculations 
avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()

            
# df build            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_name)

#formating
scores_by_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
Small (<1000),83.8,83.8,94.0%,96.0%,90.1%
Medium (1000-2000),83.4,83.4,93.6%,96.8%,90.6%
Large (>2000),77.5,77.5,68.7%,82.1%,56.6%


In [12]:
# group by type of school
by_type = school_data_complete.groupby("type")

#calculations 
avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()

# df build            
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})
    
#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]
scores_by_type.index.name = "Type of School"


#formating
scores_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.4,93.7%,96.6%,90.6%
District,77.0,77.0,66.5%,80.9%,53.7%
